In [3]:
import pandas as pd
import jsonlines
import numpy as np

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns


In [15]:
# Zero-Shot Stance Detection: A Dataset and Model using Generalized Topic Representations

csv_file = "data/vast_test.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(3)


,author,post,ori_topic,ori_id,new_topic,label,type_idx,new_id,arc_id,text,pos_text,text_s,topic,topic_str,seen?,contains_topic?,LexSim,Qte,Sarc,Imp,mlS,mlT
429,Jeff Doane,Forcing a person to do something never works. People are defiant when they are addicted to a substance and just forcing something on them can have an even greater negative effect if they aren't ready to accept they have a problem. If the person truly wants help them help should be available but not forced. When an addict has there white flag waving that they are done with that life then a program should be ready to get them into a correct treatment facility.,drug addicts,A1993-P,drug addicts,0,1,1238,16634255,"[[""forcing"", ""person"", ""something"", ""never"", ""works""], [""people"", ""defiant"", ""addicted"", ""substance"", ""forcing"", ""something"", ""even"", ""greater"", ""negative"", ""effect"", ""n't"", ""ready"", ""accept"", ""problem""], [""person"", ""truly"", ""wants"", ""help"", ""help"", ""available"", ""forced""], [""addict"", ""white"", ""flag"", ""waving"", ""done"", ""life"", ""program"", ""ready"", ""get"", ""correct"", ""treatment"", ""facility""]]","[[""VBG"", ""NN"", ""NN"", ""RB"", ""VBZ""], [""NNS"", ""JJ"", ""VBN"", ""NN"", ""VBG"", ""NN"", ""RB"", ""JJR"", ""JJ"", ""NN"", ""RB"", ""JJ"", ""VB"", ""NN""], [""NN"", ""RB"", ""VBZ"", ""VB"", ""VB"", ""JJ"", ""VBN""], [""NN"", ""JJ"", ""NN"", ""VBG"", ""VBN"", ""NN"", ""NN"", ""JJ"", ""VB"", ""JJ"", ""NN"", ""NN""]]",forcing person something never works. people defiant addicted substance forcing something even greater negative effect n't ready accept problem. person truly wants help help available forced. addict white flag waving done life program ready get correct treatment facility,"[""drug"", ""addict""]",drug addict,0,0,"[[""drug"", ""addiction""]]",0,0,1,1,1
2095,Andrew,"The Chief Justice's analogy to a baseball umpire was terrible then and it's still terrible now. Yes, judges normally call judicial balls and strikes. They accept the playing field as level and then apply the rules to whatever controversy arises between the players. But the whole point of having a Supreme Court is to decide what happens when the playing field isn't level. What happens, for example, when one of the players can't even get inside the stadium because he's black? Or when the playe...",elections,A3055-P,uneven playing fields in life,2,4,22614,19122372,"[[""chief"", ""justice"", ""'s"", ""analogy"", ""baseball"", ""umpire"", ""terrible"", ""'s"", ""still"", ""terrible""], [""yes"", ""judges"", ""normally"", ""call"", ""judicial"", ""balls"", ""strikes""], [""accept"", ""playing"", ""field"", ""level"", ""apply"", ""rules"", ""whatever"", ""controversy"", ""arises"", ""players""], [""whole"", ""point"", ""supreme"", ""court"", ""decide"", ""happens"", ""playing"", ""field"", ""n't"", ""level""], [""happens"", ""example"", ""one"", ""players"", ""ca"", ""n't"", ""even"", ""get"", ""inside"", ""stadium"", ""'s"", ""black""], [""players"", ""b...","[[""NNP"", ""NNP"", ""POS"", ""NN"", ""NN"", ""NN"", ""JJ"", ""VBZ"", ""RB"", ""JJ""], [""UH"", ""NNS"", ""RB"", ""VBP"", ""JJ"", ""NNS"", ""NNS""], [""VBP"", ""NN"", ""NN"", ""NN"", ""VB"", ""NNS"", ""WDT"", ""NN"", ""VBZ"", ""NNS""], [""JJ"", ""NN"", ""NNP"", ""NNP"", ""VB"", ""VBZ"", ""NN"", ""NN"", ""RB"", ""NN""], [""VBZ"", ""NN"", ""CD"", ""NNS"", ""MD"", ""RB"", ""RB"", ""VB"", ""IN"", ""NN"", ""VBZ"", ""JJ""], [""NNS"", ""VBP"", ""NN""], [""VBZ"", ""NN"", ""POS"", ""JJ"", ""NN"", ""NN""], [""VBZ"", ""NNP"", ""NNP"", ""NNS"", ""VBP"", ""NN"", ""VBZ"", ""VBG"", ""NN"", ""POS"", ""VB"", ""NNS"", ""NNS"", ""JJ"", ""NN""], [""VB"", ""...",chief justice 's analogy baseball umpire terrible 's still terrible. yes judges normally call judicial balls strikes. accept playing field level apply rules whatever controversy arises players. whole point supreme court decide happens playing field n't level. happens example one players ca n't even get inside stadium 's black. pl

In [16]:
# Detecting Polarized Topics Using Partisanship-aware Contextualized Topic Embeddings

target_sources = ['CNN', 'Fox News', 'Huffington Post US', 
                  'Breitbart', 'The New York Times', 'New York Post']

aylien_json_path = 'data/aylien_covid_news_data.jsonl'
articles = []
i = 0
with jsonlines.open(aylien_json_path) as reader:
    for obj in reader:
        source =  obj['source']['name']
        if source in target_sources:
            summary = obj['summary']['sentences']
            country = 'US'
            info = (obj['title'], source, obj['published_at'], obj['language'], country, tuple(summary))
            articles.append(info)
            i += 1
            if i % 2000 == 0:
                print(i, source)
data_raw = pd.DataFrame(articles, columns=['title', 'source', 'date', 'language', 'country', 'summary'])

2000 CNN
4000 CNN
6000 New York Post
8000 CNN
10000 The New York Times
12000 Breitbart
14000 The New York Times
16000 CNN
18000 Fox News
20000 CNN
22000 CNN
24000 The New York Times
26000 Fox News
28000 CNN
30000 Huffington Post US
32000 CNN
34000 CNN
36000 Fox News
38000 Fox News
40000 Fox News
42000 New York Post
44000 CNN
46000 Huffington Post US
48000 CNN
50000 Fox News
52000 CNN
54000 New York Post
56000 New York Post
58000 CNN
60000 New York Post
62000 Huffington Post US
64000 CNN
66000 The New York Times
68000 Fox News
70000 CNN
72000 Fox News
74000 New York Post


In [17]:
import copy
data = copy.deepcopy(data_raw)
data.sample(5)

,title,source,date,language,country,summary
3229,Opinion | Covid-19 Is Twisting 2020 Beyond All Recognition,The New York Times,2020-04-01 09:04:08+00:00,en,US,"(Paul Conway, a professor of psychology at Florida State University, argued that “evidence from history and the psychological literature suggests that under times of stress and uncertainty, there are a number of different effects.”, There is, however, another side of the coin, Conway said:\nWhen times are more uncertain and threatening, then strangers and outsiders and people different from oneself feel more threatening on an intuitive, gut level., Along parallel lines, Conway argued,\nthe t..."
44954,Shop these 31 brands that are giving back to coronavirus relief efforts,CNN,2020-04-21 18:37:50+00:00,en,US,"(This shoe brand has promised to donate 10% of proceeds from its ""Share the Love"" Collection to Feeding America, which supports crucial food banks across the country., The baby fashionwear brand will donate 30% of proceeds from every purchase directly to NoKidHungry.com along with an option for direct donation without purchase\nThis phone case and screen protector brand is dishing out 10% off all purchases with code BETTERTOGETHER, and donating 10% of those BETTERTOGETHER proceeds to Relief..."
18386,Louisiana reports first presumptive positive case of coronavirus,CNN,2020-03-09 20:53:03+00:00,en,US,"(The Louisiana Department of Health has reported the state's first presumptive positive case of coronavirus, according to a statement from Gov. John Bel Edwards' office., ""While today is the first time that we can confirm that we have a presumptive positive coronavirus case, Louisiana has been preparing for this moment for many weeks,"" Edwards said., The governor will hold a news conference with public health officials later today., In the US, as of Monday, coronavirus was reported in 35 sta..."
51742,UFC 249 will not happen April 18 and all other events are postponed indefinitely,CNN,2020-04-10 05:46:21+00:00,en,US,"(Additionally, all other UFC events have been postponed indefinitely., ""While the organization was fully prepared to proceed with UFC 249, ESPN has requested the postponement of the event and subsequent bouts until further notice in light of the Covid-19 pandemic,"" a UFC spokesperson said in a statement to CNN on Thursday., On Thursday, Sen. Dianne Feinstein said in a statement that the event should be postponed., ""I'm concerned by reports that Ultimate Fighting Championship plans to hold a ..."
18272,SEC becomes first federal agency to ask DC employees to work from home over potential coronavirus case,CNN,2020-03-10 02:20:50+00:00,en,US,"(The Securities and Exchange Commission is asking all personnel based at its headquarters to work from home due to an employee who may have coronavirus, according to an SEC spokesperson, becoming the first federal agency to ask its Washington employees to stay home., ""Late this afternoon, the SEC was informed that a Washington, DC Headquarters employee was treated for respiratory symptoms today (Monday),"" the spokesperson said in a statement., ""The employee was informed by a physician that t..."


In [18]:
# Analyzing Political Bias and Unfairness in News Articles at Different Levels of Granularity
# Political Ideology and Polarization: A Multi-dimensional Approach

import json

with open("data/released_data.json", "r") as file:
    j_string = file.readline()
    jsons = []
    while j_string:
        data = json.loads(j_string)
        jsons.append(data)
        j_string = file.readline()
        
df = pd.DataFrame(jsons)

df.sample()

,source,title,event_id,adfontes_fair,adfontes_political,allsides_bias,content,misc
2946,The Hill,Five things to watch for in Trump’s tax plan,0,bias,neutral,From the Center,"President Trump is set to release his tax plan on Wednesday.\nThe announcement of the administration's principles for tax reform will begin a new debate with Congress as the president nears the end of his first 100 days in office.\nIt appears Trump will leave the actual reveal to Treasury Secretary Steven Mnuchin and National Economic Director Gary Cohn, who will be at the White House press briefing on Wednesday afternoon.\nThey’re slated to meet the press at the same time Trump has a schedu...","{'time': '2017-04-26 06:00:00-04:00', 'topics': 'Taxes', 'author': ['Naomi Jagoda']}"


In [19]:
# Detecting White Supremacist Hate Speech Using Domain Specific Word Embedding With Deep Learning and BERT (48 citations)
csv_file = "data/WhiteSupremacistDataset.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(2)

,input.text,annot1,annot2,annot3,Coded Labels,Coded Labels.1,Coded Labels.2,Binary Coded Labels,Binary Coded Labels.1,Binary Coded Labels.2,Voting and Final Labels
682,TweetOfTheDay,Neutral,Neutral,Neutral,0,0,0,0,0,0,0
628,Aight Assad boys it rape time,Other hate Speech,Neutral,Implicit White Supremacism,3,0,1,1,0,1,1


In [20]:
# Automated Hate Speech Detection and the Problem of Offensive Language (2201 citations)

csv_file = "data/labeled_data.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
17886,18288,3,0,0,3,2,RT @TwitchyTeam: Richard Grenell: WaPo's Greg Sargent mocks voter ID and 'proves point' of proponents http://t.co/KG3NEWOYIy
22814,23293,3,1,2,0,1,Who tf make an account of another bitch... Like you wanna be anotha bitch dat bad.. Lmfaooo fat ass hoes
9185,9441,6,0,6,0,1,Free lil cj bitch the wait is over ! #december8th
18298,18705,3,0,3,0,1,"RT @_JohnFKennedy_: I choose to fuck all these bitches not because they're easy, but because I'm hard #PresidentialPounding"
19985,20427,3,0,2,1,1,"RT @scott1023: ""I said thin mints bitch"". http://t.co/yRVQjxeyqO"


In [21]:
# Political Polarization During the COVID-19 Pandemic (51 citations)
csv_file = "data/corona_comp_1720.dta"
df = pd.read_stata(csv_file)

# Display the DataFrame
df.sample(5)

,Vignette,edu2,sex,age,income,cducsu,spd,fdp,green,left,afd,other,dnv,feel,group,vote,time
4548,13.0,low,female,69.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,ITA,4,2017
2068,9.0,high,male,43.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,CDU,NaN,2020-W2
2605,13.0,high,male,43.0,9.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,39.0,EU,3,2020-W2
2384,14.0,low,female,46.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50.0,Nationalist,7,2020-W2
4462,9.0,high,female,42.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,CDU,NaN,2017


In [22]:
# Knowledge Enhanced Masked Language Model for Stance Detection

csv_file = "data/2020_twitter/biden_stance_test_public.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,tweet_id,text,label
12,1291065309063938048,joe #biden kept saying to the \namerican people that he is\nready to #debate president #trump \nand accepted 3 #three debates \ndoes joe #biden contemplate\nhis intention to #refuse \nand #avoid these debates\non a #strategy using the #coronavirus \nfor not wanting to travel \n#prediction,NONE
273,1304630729117233153,"that 1977 @USER is one loving #blm supporter alright. and @USER isn’t responsible for her grandparents owning slaves; then no one else is either, right? well that’s how equality works... 🤷🏼‍♂️ #trump2020 HTTP",AGAINST
15,1296228387393077249,"@USER see #2a supporters, the #gop is coming for your guns. ""*400 firearms seized by atf."" rise up! defend #2a!!!!!",NONE
251,1296601660475809793,"joe biden “has to make a positive case and present a bigger vision than ‘i am not trump, let’s go back to normal,’” former presidential candidate @USER said in april. \nyang will be speaking at the #dnc tonight.\nHTTP",NONE
152,1296621905127059456,@USER tribute 😭😢@USER @USER gave me chills! 💝 #glory ✊🏼👏🏻👏🏻👏🏻🥰#powerofthepeople #demconvention #democrats #hopeful #bidenharris2020tosaveamerica #bidenharrislandslide2020,FAVOR


In [23]:
# Knowledge Enhanced Masked Language Model for Stance Detection

csv_file = "data/2020_twitter/trump_stance_test_public.csv"
df = pd.read_csv(csv_file)

# Display the DataFrame
df.sample(5)

,tweet_id,text,label
294,1310964026881474561,@USER make sure that sleepy joe biden has drugs test .that cnnn and nytimess fake news get some draper to its pervert and corrupt candidate joe biden. joe biden china puppet imagine a candidate with a humans trafficking son hunter biden,FAVOR
73,1296234548972453889,@USER @USER @USER @USER sounds just like me. i’ll give you some good advice on a good team to follow. feel free to keep the below picture. go big orange! #kag HTTP,FAVOR
6,1267949626080059392,behind their fake #democratic agenda. this was the plan all along. it’s real it’s happening and don’t think for a minute that these fifty united states couldn’t get thrust into another civil war. the name of the game is the same! freedom versus tyranny,NONE
272,1289029134212124672,"republicans will let the postal service close &amp; with the #coronavirus still raging, it's the ultimate in voter suppression. \n#mailinballots must be allowed, how do we stop this from happening?\n#vote #voteblue2020\n\nHTTP",AGAINST
263,1225943140953182208,you don’t realize what a big deal the #demdebate and the nh primary is for the only commercial tv station in the state until you follow @USER twitter feed.,NONE
